In [37]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from functions import VM
import os
import numpy as np

In [38]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# Take the sign names in our collections data

In [13]:
signs = os.listdir("Model_data") # take all the signs used in data collection
sign_id = {label: num for num, label in enumerate(signs)} # assign a value for each sign (each sign is assign to its index)

# Create the our x and y data


In [14]:
sequences, labels = [], [] # sequences represents our x data while labels represents our y data
# create a loop that creates our x and y data
for sign in signs:
    for sequence in range(VM.seq):
        window = []
        for frames in range(VM.seq_lenght):
            res = np.load(os.path.join(VM.DATA_PATH, sign, str(sequence), f"{frames}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(sign_id[sign])
        
# store our x and y data to a numpy array
x_data = np.array(sequences) 
y_data = to_categorical(labels).astype(int) 

# setup our training and testing partition
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

# Build and train the model


In [35]:
# build the model
model = Sequential()
model.add(LSTM(64, return_sequences="True", activation="relu", input_shape=(30,1662)))
model.add(LSTM(128, return_sequences="True", activation="relu"))
model.add(LSTM(64, return_sequences="False", activation="relu"))
model.add(Flatten(input_shape=(x_train.shape[1:])))
model.add(Dense(32, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(np.array(signs).shape[0], activation="softmax"))

In [40]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]) # compiler 

In [41]:
model.fit(x_train, y_train, epochs=2000) # run this line when data collection is done (this will train the model)
# we can start the number of epochs to be equal to 175 to avoid over training and add afterwards if needed

Epoch 1/2000


ValueError: in user code:

    File "C:\Users\Baron\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Baron\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Baron\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Baron\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Baron\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Baron\A

In [ ]:
model.save("VertoMotus_MLmodel") # save model